In [29]:
from pulp import * 
import pandas as pd
df = X = pd.DataFrame(pd.read_csv("city.csv"))
df.head()

,X,Y,FID,NAME,CLASS,ST,STFIPS,PLACEFIPS,CAPITAL,POP_CLASS,...,MARHH_CHD,MARHH_NO_C,MHH_CHILD,FHH_CHILD,FAMILIES,AVE_FAM_SZ,HSE_UNITS,VACANT,OWNER_OCC,RENTER_OCC
0,-74.010132,40.710571,2174,New York,city,NY,36,3651000,,10,...,531386,566484,77061,344755,1850221,3.32,3371062,261278,962892,2146892
1,-118.270582,34.052790,415,Los Angeles,city,CA,6,644000,,10,...,277289,245056,46908,118939,807326,3.53,1413995,95827,503863,814305
2,-87.647146,41.756488,63,Chicago,city,IL,17,1714000,,10,...,165868,168435,28332,116555,576793,3.40,1194337,148777,469562,575998
3,-95.367507,29.758761,3641,Houston,city,TX,48,4835000,,10,...,159610,145478,26039,85014,481570,3.38,892646,110003,355236,427407
4,-112.073868,33.446114,910,Phoenix,city,AZ,4,455000,State,10,...,115852,102047,23353,54452,330762,3.42,590149,75343,296742,218064


In [30]:
dfsmall=df.filter(items=['NAME', 'AGE_10_14', 'AGE_15_19','AGE_20_24', 'AGE_25_34', 'MALES', 'MED_AGE_M','POPULATION', 'POP2010','MED_AGE','AVE_FAM_SZ'], axis=1).filter(items=range(0,20), axis=0)
dfsmall.columns=('city','age10_14','age15_19','age20_24', 'age25_34', 'male', 'mid_age_male', 'population', 'pop2010', 'midage', 'avg_fm_size')
dfsmall

,city,age10_14,age15_19,age20_24,age25_34,male,mid_age_male,population,pop2010,midage,avg_fm_size
0,New York,468154,535833,642585,1392445,3882544,34.1,8691599,8175133,35.5,3.32
1,Los Angeles,237462,274373,314543,638900,1889064,33.3,3986442,3792621,34.2,3.53
2,Chicago,164466,182933,223027,515551,1308072,32.4,2781116,2695598,33.1,3.40
3,Houston,137307,142544,171086,373985,1053517,31.5,2333285,2099451,32.2,3.38
4,Phoenix,108732,110352,106757,224128,725020,31.6,1601381,1445632,32.2,3.42
5,Philadelphia,90640,118297,146717,246062,719813,32.1,1587761,1526006,33.6,3.20
6,San Antonio,97421,102486,107623,197161,647690,31.4,1442472,1327407,32.8,3.34
7,San Diego,75345,90813,127987,230247,660626,32.8,1397856,1307402,33.7,3.28
8,Dallas,78234,76976,94880,219848,598962,31.4,1323651,1197816,31.9,3.42
9,San Jose,61881,62973,64386,145310,475668,34.4,1042940,945942,35.2,3.54


In [31]:
prob = LpProblem(name="Warehouse", sense=LpMaximize)

data = pulp.LpVariable.dicts('city',(name for name in dfsmall['city']), lowBound=0, upBound=1, cat='Binary')

### First LP

In [32]:
#objective
prob += lpSum([data[dfsmall.loc[i,'city']]*(int(dfsmall.loc[i,'age20_24'])+int(dfsmall.loc[i,'age25_34'])) for i in dfsmall.index])

#contraints
prob += lpSum([data[dfsmall.loc[i,'city']]*int(dfsmall.loc[i,'male']) for i in dfsmall.index]) >= 120000
prob += lpSum([data[dfsmall.loc[i,'city']]*int(dfsmall.loc[i,'mid_age_male']) for i in dfsmall.index]) <= 124
prob += lpSum([data[dfsmall.loc[i,'city']]*int(dfsmall.loc[i,'age10_14']) for i in dfsmall.index]) >= 200000
prob += lpSum([data[dfsmall.loc[i,'city']]*int(dfsmall.loc[i,'age15_19']) for i in dfsmall.index]) >= 120000

prob += lpSum([data[dfsmall.loc[i,'city']] for i in range(1,20)]) == 4


In [33]:
prob.solve()
LpStatus[prob.status]

'Optimal'

In [35]:
for i in data:
    print(data[i], ': ', data[i].varValue)

city_New_York :  0.0
city_Los_Angeles :  1.0
city_Chicago :  0.0
city_Houston :  1.0
city_Phoenix :  0.0
city_Philadelphia :  0.0
city_San_Antonio :  0.0
city_San_Diego :  0.0
city_Dallas :  0.0
city_San_Jose :  0.0
city_Austin :  1.0
city_Jacksonville :  0.0
city_Columbus :  1.0
city_San_Francisco :  0.0
city_Indianapolis :  0.0
city_Fort_Worth :  0.0
city_Charlotte :  0.0
city_Denver :  0.0
city_El_Paso :  0.0
city_Seattle :  0.0


In [36]:
value(prob.objective)

1982741.0

### Cities we choose: Los Angeles, Houston, Austin, Columbus

### Second LP

In [37]:
prob = LpProblem(name="Warehouse", sense=LpMaximize)
data = pulp.LpVariable.dicts('city',(name for name in dfsmall['city']), lowBound=0, upBound=1, cat='Binary')

In [38]:
#objective
prob += lpSum([data[dfsmall.loc[i,'city']]*(int(dfsmall.loc[i,'population'])/int(dfsmall.loc[i,'pop2010'])) for i in dfsmall.index])

#contraints
prob += lpSum([data[dfsmall.loc[i,'city']]*int(dfsmall.loc[i,'midage']) for i in dfsmall.index]) <= 130
prob += lpSum([data[dfsmall.loc[i,'city']]*int(dfsmall.loc[i,'avg_fm_size']) for i in dfsmall.index]) >= 12
prob += lpSum([data[dfsmall.loc[i,'city']]*int(dfsmall.loc[i,'population']) for i in dfsmall.index]) >= 2000000
prob += lpSum([data[dfsmall.loc[i,'city']]*int(dfsmall.loc[i,'age15_19']) for i in dfsmall.index]) >= 120000

prob += lpSum([data[dfsmall.loc[i,'city']] for i in range(1,20)]) == 4


In [39]:
prob.solve()
LpStatus[prob.status]

'Optimal'

In [40]:
for i in data:
    print(data[i], ': ', data[i].varValue)

city_New_York :  0.0
city_Los_Angeles :  0.0
city_Chicago :  0.0
city_Houston :  0.0
city_Phoenix :  0.0
city_Philadelphia :  0.0
city_San_Antonio :  0.0
city_San_Diego :  0.0
city_Dallas :  0.0
city_San_Jose :  0.0
city_Austin :  1.0
city_Jacksonville :  0.0
city_Columbus :  0.0
city_San_Francisco :  0.0
city_Indianapolis :  0.0
city_Fort_Worth :  1.0
city_Charlotte :  1.0
city_Denver :  1.0
city_El_Paso :  0.0
city_Seattle :  0.0


In [41]:
value(prob.objective)

4.644883461298303

### Cities we choose: Austin, Fort Worth, Charlotte, Denver